In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Groceries-data-train.csv')
df.head(5)


,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1


In [3]:
df_test = pd.read_csv('Groceries-data-test.csv')
df_test.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3481,8/03/2015,candy,2015,3,8,6
1,1254,19/04/2015,white wine,2015,4,19,6
2,2835,28/01/2014,domestic eggs,2014,1,28,1
3,2854,2/08/2015,coffee,2015,8,2,6
4,4637,12/08/2014,bottled water,2014,8,12,1


In [4]:
items = df['itemDescription'].unique()
items.shape


(167,)

In [5]:
items = df[(df['Member_number'] == 3021) & (df['Date'] == "30/01/2015")] ["itemDescription"]
print(items)

0          frankfurter
21033    sliced cheese
Name: itemDescription, dtype: object


In [6]:
member_df = df.groupby(['Member_number', 'Date'], as_index=False).agg({
    'itemDescription': ', '.join,
    'year': 'first',  # Retain the year (or use any other appropriate method)
    'month': 'first',  # Retain the month
    'day': 'first',  # Retain the day
    'day_of_week': 'first'  # Retain the day of the week
})
member_df.head(5)



,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1000,15/03/2015,"sausage, yogurt",2015,3,15,6
1,1000,24/06/2014,pastry,2014,6,24,1
2,1000,24/07/2015,"misc. beverages, canned beer",2015,7,24,4
3,1000,25/11/2015,sausage,2015,11,25,2
4,1000,27/05/2015,"soda, pickled vegetables",2015,5,27,2


In [7]:
member_df_test = df_test.groupby(['Member_number', 'Date'], as_index=False).agg({
    'itemDescription': ', '.join,
    'year': 'first',  # Retain the year (or use any other appropriate method)
    'month': 'first',  # Retain the month
    'day': 'first',  # Retain the day
    'day_of_week': 'first'  # Retain the day of the week
})
member_df_test.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1000,15/03/2015,"semi-finished bread, whole milk",2015,3,15,6
1,1000,24/06/2014,"whole milk, salty snack",2014,6,24,1
2,1000,25/11/2015,hygiene articles,2015,11,25,2
3,1001,20/01/2015,frankfurter,2015,1,20,1
4,1002,2/09/2014,other vegetables,2014,9,2,1


In [8]:
import pandas as pd

# Group by Member_number and itemDescription, then count the occurrences
purchase_counts = df.groupby(['Member_number', 'itemDescription']).size().reset_index(name='Purchase_Count')


# Initialize an empty DataFrame with Member_number as rows and itemDescription as columns
members = df['Member_number'].unique()
items = df['itemDescription'].unique()

new_df = pd.DataFrame(index=members, columns=items).fillna(0)

# Populate the new DataFrame with purchase counts
for _, row in purchase_counts.iterrows():
    member = row['Member_number']
    item = row['itemDescription']
    count = row['Purchase_Count']
    new_df.at[member, item] = count

# Reset the index to make Member_number a column
new_df = new_df.reset_index().rename(columns={'index': 'Member_number'})



In [9]:
new_df.set_index('Member_number', inplace=True)


In [10]:
new_df = new_df.sort_index()
new_df.shape


(3872, 167)

TF-IDF

In [11]:
tf = new_df.copy()


# Count the number of 0s in each row
tf = tf.apply(lambda row: row / (167 - (row == 0).sum()), axis=1)

tf.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0.000000,0.0,0.0,0.0,0.0,0.0,0.142857,0.285714,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.111111,0.111111,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.000000,0.0,0.2,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.076923,0.076923,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
columns = tf.columns
idf = []
for i in columns:
  idf.append(np.log10(3872 / (tf[i]!=0).sum())) # number of member / number of member purchse this item
print(idf)

[0.9935427982609293, 1.0220875299628385, 0.7710315092606957, 1.2575215752871651, 1.3114735444631118, 2.08278537031645, 0.6864770272402437, 0.8290234562383825, 0.5797611646299297, 1.1970002415329768, 0.9444826721501687, 1.0268339649873, 1.0488592498435794, 1.33266284353305, 0.4592130642979293, 0.78997570489916, 1.3302567737671716, 1.9646860582384555, 0.5453598361961655, 1.0451099216771762, 1.3600486440226824, 1.1166436375774174, 2.1565715844773687, 1.051376906064826, 1.1917360015406198, 1.5348569051529364, 1.211358391579844, 1.3574864272580822, 1.0092961386682837, 2.5087541025887314, 1.2415823741857175, 1.9347228348610124, 1.6102117433475083, 1.0047365746677333, 1.3574864272580822, 1.0451099216771762, 2.20772410692475, 1.8245073550734188, 0.6288939563152625, 2.1108140939166935, 1.4176736332413986, 2.33266284353305, 0.9913382530098959, 1.4206180138881799, 2.2657160539024366, 0.9270698706324869, 0.9356890076330329, 1.2376873303021931, 2.0564564315941007, 2.08278537031645, 0.91954943194635

In [14]:
tf_idf = tf.apply(lambda row: row * idf, axis=1)
tf_idf.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.098068,0.236864,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.110394,0.0,0.000000,0.0,0.0,0.0,0.000000,0.092114,0.064418,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.000000,0.0,0.154206,0.0,0.0,0.0,0.000000,0.165805,0.115952,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.044597,0.092077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
# Melt the DataFrame to have a long format DataFrame with columns 'variable' and 'value'
a=[]
for i in range(3872):
  row = new_df.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  b = []
  for ii in range(5):
      b.append((top_5_columns[ii], top_5_values[ii]))
  a.append(b)
print(a)

[[('sausage', 2), ('yogurt', 1), ('misc. beverages', 1), ('soda', 1), ('canned beer', 1)], [('whole milk', 2), ('soda', 2), ('frankfurter', 1), ('sausage', 1), ('rolls/buns', 1)], [('butter', 1), ('whole milk', 1), ('tropical fruit', 1), ('sugar', 1), ('frozen vegetables', 1)], [('root vegetables', 1), ('sausage', 1), ('rolls/buns', 1), ('dental care', 1), ('detergent', 1)], [('whole milk', 3), ('rolls/buns', 1), ('chocolate', 1), ('other vegetables', 1), ('red/blush wine', 1)], [('rolls/buns', 2), ('whipped/sour cream', 1), ('margarine', 1), ('frankfurter', 0), ('pork', 0)], [('frankfurter', 2), ('rolls/buns', 1), ('chocolate', 1), ('bottled beer', 1), ('shopping bags', 1)], [('yogurt', 1), ('tropical fruit', 1), ('dessert', 1), ('domestic eggs', 1), ('hamburger meat', 1)], [('yogurt', 1), ('tropical fruit', 1), ('pastry', 1), ('herbs', 1), ('canned fish', 1)], [('pip fruit', 2), ('bottled water', 2), ('coffee', 1), ('UHT-milk', 1), ('sliced cheese', 1)], [('frankfurter', 1), ('rolls/

In [177]:
c = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = a)
c.head(50)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(sausage, 2)","(yogurt, 1)","(misc. beverages, 1)","(soda, 1)","(canned beer, 1)"
1001,"(whole milk, 2)","(soda, 2)","(frankfurter, 1)","(sausage, 1)","(rolls/buns, 1)"
1002,"(butter, 1)","(whole milk, 1)","(tropical fruit, 1)","(sugar, 1)","(frozen vegetables, 1)"
1003,"(root vegetables, 1)","(sausage, 1)","(rolls/buns, 1)","(dental care, 1)","(detergent, 1)"
1004,"(whole milk, 3)","(rolls/buns, 1)","(chocolate, 1)","(other vegetables, 1)","(red/blush wine, 1)"
1005,"(rolls/buns, 2)","(whipped/sour cream, 1)","(margarine, 1)","(frankfurter, 0)","(pork, 0)"
1006,"(frankfurter, 2)","(rolls/buns, 1)","(chocolate, 1)","(bottled beer, 1)","(shopping bags, 1)"
1008,"(yogurt, 1)","(tropical fruit, 1)","(dessert, 1)","(domestic eggs, 1)","(hamburger meat, 1)"
1009,"(yogurt, 1)","(tropical fruit, 1)","(pastry, 1)","(herbs, 1)","(canned fish, 1)"


In [20]:
new_df_columns = new_df.columns
print(new_df_columns)

Index(['frankfurter', 'pork', 'root vegetables', 'onions', 'waffles',
       'cereals', 'yogurt', 'sausage', 'rolls/buns', 'chocolate',
       ...
       'organic sausage', 'house keeping products', 'frozen fruits',
       'bathroom cleaner', 'skin care', 'bags', 'rubbing alcohol',
       'make up remover', 'preservation products', 'kitchen utensil'],
      dtype='object', length=167)


UV decomposition

In [18]:
import numpy as np

def uv_decomposition(R, k, learning_rate, regularization):
    """
    Performs UV decomposition on the input matrix R, with a target rank of k, using stochastic gradient descent (SGD).
    Returns the decomposed matrices U and V.
    """
    # Initialize U and V with random values
    num_users, num_items = R.shape
    U = np.random.rand(num_users, k)
    V = np.random.rand(k, num_items)

    # Perform stochastic gradient descent to optimize U and V
    for epoch in range(10):
        for i in range(num_users):
            for j in range(num_items):
                if R[i, j] > 0:
                    error = R[i, j] - np.dot(U[i, :], V[:, j])
                    U[i, :] += learning_rate * (error * V[:, j] - regularization * U[i, :])
                    V[:, j] += learning_rate * (error * U[i, :] - regularization * V[:, j])

    # Return the decomposed matrices U and V
    return U, V

In [181]:
numpy_array = new_df.to_numpy()
print(numpy_array)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [182]:
def calculate_rmse(R, U, V):
    predicted_R = np.dot(U, V)
    error = R - predicted_R
    error = error[R > 0]  # Only consider known values
    return np.sqrt(np.mean(error**2))

k_values = [1,2,3,4,5,6]

# Split the data into training and validation sets

best_k = None
best_rmse = float('inf')

for k in k_values:
    U, V = uv_decomposition(numpy_array, k, 0.1, 0.1)
    rmse = calculate_rmse(numpy_array, U, V)
    print(f"k={k}, RMSE={rmse}")
    if rmse < best_rmse:
        best_rmse = rmse
        best_k = k

print(f"Best k: {best_k} with RMSE: {best_rmse}")

k=1, RMSE=0.2988032474164601
k=2, RMSE=0.2981750719116172
k=3, RMSE=0.2970775101806233
k=4, RMSE=0.29760003899692744
k=5, RMSE=0.2973134436647349
k=6, RMSE=0.2984813349223511
Best k: 3 with RMSE: 0.2970775101806233


In [183]:
# Perform UV decomposition on R
U, V = uv_decomposition(numpy_array, k=3, learning_rate=0.1, regularization=0.1)

# Reconstruct the matrix R using the decomposed matrices U and V
numpy_array_reconstructed = np.dot(U, V)


In [184]:

df_reconstructed = pd.DataFrame(numpy_array_reconstructed, columns=new_df_columns)
df_reconstructed.index = new_df.index
df_reconstructed.shape

(3872, 167)

In [185]:
new_df.head(5)


,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [186]:
# Melt the DataFrame to have a long format DataFrame with columns 'variable' and 'value'
z=[]
for i in range(3872):
  row = df_reconstructed.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  x = []
  for ii in range(5):
      x.append((top_5_columns[ii], top_5_values[ii]))
  z.append(x)
print(z)

[[('margarine', 1.2475254588648776), ('yogurt', 1.222021322907811), ('other vegetables', 1.169489705316581), ('sausage', 1.1547136017921953), ('soda', 1.0941426321606706)], [('margarine', 1.4283025211967226), ('yogurt', 1.3556806898968197), ('other vegetables', 1.3304317411822792), ('sausage', 1.287350626058197), ('pip fruit', 1.252891359492609)], [('margarine', 1.20642406731803), ('other vegetables', 1.1503213678406454), ('yogurt', 1.1392932533270235), ('sausage', 1.1132762955559863), ('rolls/buns', 1.0965290216365384)], [('margarine', 1.1252692015375931), ('yogurt', 1.0750143694872465), ('other vegetables', 1.0390981850246839), ('sausage', 1.0082839577026061), ('bags', 1.0079596243139317)], [('margarine', 1.2653477424450357), ('yogurt', 1.209915796698602), ('other vegetables', 1.1775405296088934), ('sausage', 1.1444483705767667), ('whole milk', 1.1047746868315775)], [('margarine', 1.326882545345771), ('other vegetables', 1.2651231060559494), ('yogurt', 1.236295766988565), ('rolls/bun

In [187]:
member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = z)
member_recomendation.head(50)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(margarine, 1.2475254588648776)","(yogurt, 1.222021322907811)","(other vegetables, 1.169489705316581)","(sausage, 1.1547136017921953)","(soda, 1.0941426321606706)"
1001,"(margarine, 1.4283025211967226)","(yogurt, 1.3556806898968197)","(other vegetables, 1.3304317411822792)","(sausage, 1.287350626058197)","(pip fruit, 1.252891359492609)"
1002,"(margarine, 1.20642406731803)","(other vegetables, 1.1503213678406454)","(yogurt, 1.1392932533270235)","(sausage, 1.1132762955559863)","(rolls/buns, 1.0965290216365384)"
1003,"(margarine, 1.1252692015375931)","(yogurt, 1.0750143694872465)","(other vegetables, 1.0390981850246839)","(sausage, 1.0082839577026061)","(bags, 1.0079596243139317)"
1004,"(margarine, 1.2653477424450357)","(yogurt, 1.209915796698602)","(other vegetables, 1.1775405296088934)","(sausage, 1.1444483705767667)","(whole milk, 1.1047746868315775)"
1005,"(margarine, 1.326882545345771)","(other vegetables, 1.2651231060559494)","(yogurt, 1.236295766988565)","(rolls/buns, 1.2243118408479106)","(sausage, 1.2146153861359226)"
1006,"(margarine, 1.248675551056738)","(yogurt, 1.2107397750873552)","(other vegetables, 1.1669345179073987)","(sausage, 1.144545922825744)","(whole milk, 1.0871840247526336)"
1008,"(margarine, 1.1714962527284603)","(yogurt, 1.1036722085911144)","(other vegetables, 1.1029489861768496)","(sausage, 1.0640019826686529)","(rolls/buns, 1.0450812849936482)"
1009,"(margarine, 1.1376897498757912)","(yogurt, 1.0634778664884577)","(other vegetables, 1.0520873825086567)","(pip fruit, 1.0321338149967763)","(bags, 1.0142280946022795)"


tf_idf combined with UV decomposition

In [21]:
tfidf_numpy_array = tf_idf.to_numpy()
# Perform UV decomposition on R
U_, V_ = uv_decomposition(tfidf_numpy_array, k=3, learning_rate=0.1, regularization=0.1)

# Reconstruct the matrix R using the decomposed matrices U and V
tfidf_numpy_array_reconstructed = np.dot(U_, V_)
df_tfidf = pd.DataFrame(tfidf_numpy_array_reconstructed, columns=new_df_columns)
df_tfidf.index = new_df.index
df_tfidf.head(5)

,frankfurter,pork,root vegetables,onions,waffles,cereals,yogurt,sausage,rolls/buns,chocolate,...,organic sausage,house keeping products,frozen fruits,bathroom cleaner,skin care,bags,rubbing alcohol,make up remover,preservation products,kitchen utensil
Member_number,,,,,,,,,,,,,,,,,,,,,
1000,0.083875,0.128893,0.098602,0.124276,0.127570,0.171502,0.076450,0.092940,0.064208,0.092685,...,0.166602,0.173572,0.295238,0.270568,0.134653,0.235288,0.163001,0.229916,0.481908,0.335897
1001,0.060826,0.092113,0.070909,0.076831,0.093366,0.125172,0.049419,0.072652,0.051850,0.067414,...,0.140665,0.128079,0.231383,0.182242,0.101243,0.089814,0.053175,0.071860,0.312787,0.178117
1002,0.120660,0.177391,0.137556,0.149666,0.190023,0.233535,0.097485,0.138342,0.098462,0.133685,...,0.307939,0.241102,0.647318,0.335018,0.201937,0.224538,0.192107,0.193230,0.552537,0.414660
1003,0.087636,0.135128,0.103408,0.122372,0.132650,0.183249,0.076322,0.101720,0.071385,0.096973,...,0.179243,0.185837,0.273564,0.280544,0.142625,0.185151,0.109564,0.169890,0.497460,0.296053
1004,0.044262,0.064067,0.049852,0.055795,0.070656,0.082371,0.036316,0.048905,0.034573,0.049008,...,0.117016,0.085381,0.276420,0.118706,0.073914,0.101488,0.096565,0.092512,0.192042,0.172760


In [22]:
a=[]
for i in range(3872):
  row = df_tfidf.iloc[i]
  top_5_values = row.nlargest(5)
  top_5_columns = top_5_values.index
  b = []
  for ii in range(5):
      b.append((top_5_columns[ii], top_5_values[ii]))
  a.append(b)
tfidf_member_recomendation = pd.DataFrame(index=new_df.index, columns=["top_1", "top_2", "top_3", "top_4", "top_5"], data = a)
tfidf_member_recomendation.head(50)

,top_1,top_2,top_3,top_4,top_5
Member_number,,,,,
1000,"(cream, 0.5049847559390475)","(preservation products, 0.4819075394702281)","(cooking chocolate, 0.4224595548919375)","(organic products, 0.3963147086264364)","(frozen chicken, 0.35900915491475766)"
1001,"(organic products, 0.402667215530065)","(preservation products, 0.3127873960855144)","(cream, 0.29489477778880674)","(baby cosmetics, 0.26934518452991096)","(nut snack, 0.2524582462033098)"
1002,"(frozen fruits, 0.6473182683828074)","(preservation products, 0.5525374456293894)","(organic products, 0.5456735284491745)","(cream, 0.5131410937686841)","(baby cosmetics, 0.5105421667658097)"
1003,"(organic products, 0.5457986568994893)","(cream, 0.5013115512110368)","(preservation products, 0.49746032698784226)","(baby cosmetics, 0.36096183458826636)","(cooking chocolate, 0.3570928605897162)"
1004,"(frozen fruits, 0.2764197329308291)","(cooking chocolate, 0.19845778442840092)","(preservation products, 0.19204247248103762)","(cream, 0.1806204878892345)","(baby cosmetics, 0.1776849318261889)"
1005,"(cream, 0.9754309891253913)","(organic products, 0.972595473308297)","(preservation products, 0.963235488394236)","(cooking chocolate, 0.7448759681048348)","(baby cosmetics, 0.6959517514082955)"
1006,"(cream, 0.43469091981188035)","(preservation products, 0.4248143495800264)","(cooking chocolate, 0.375034532112721)","(organic products, 0.3552516777121199)","(frozen fruits, 0.3192079023139135)"
1008,"(frozen fruits, 0.42041860863401537)","(organic products, 0.38388829762331056)","(baby cosmetics, 0.335759856455661)","(preservation products, 0.32085585472318856)","(nut snack, 0.31515572686842847)"
1009,"(organic products, 0.6719622852635742)","(preservation products, 0.5176482834608728)","(cream, 0.48474035076312827)","(baby cosmetics, 0.45303235046718887)","(nut snack, 0.42428137763543744)"


Simple

In [ ]:
user1 = new_df.drop(columns = ["Member_number"]).iloc[1].to_numpy()
user2 = new_df.drop(columns = ["Member_number"]).iloc[3].to_numpy()
def similarity(user1, user2):
    # Calculate the dot product of the two user vectors
    dot_product = np.dot(user1, user2)
    print(dot_product)
    # Calculate the magnitude of the two user vectors
    magnitude = np.sqrt(np.sum(user1 ** 2) * np.sum(user2 ** 2))
    # Calculate the similarity between the two users
    similarity = dot_product / magnitude
    return similarity
x = similarity(user1, user2)
print(x)
print(user1)
print(user2)

1
0.22360679774997896
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
